In [127]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import copy
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os
import pandas as pd
import pickle
from scipy.signal import decimate
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean, plot_one_side, calculate_error_bars
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.individual_trial_analysis_utils import ZScoredTraces, SessionData, CueAlignedData
from utils.post_processing_utils import *
import seaborn as sns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
mouse_id = 'SNL_photo21'
date = '20200915'
all_experiments = get_all_experimental_records()
experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
session_data, trial_data = open_experiment(experiment_to_process)

In [60]:
params = {'state_type_of_interest': 5,
    'outcome': 2,
    'last_outcome': 0,  # NOT USED CURRENTLY
    'no_repeats' : 0,
    'last_response': 0,
    'align_to' : 'Time start',
    'instance': -1,
    'plot_range': [-6, 6],
    'first_choice_correct': 0,
     'cue': 'None'}
test = CustomAlignedData(session_data, params)

(236, 160000)
(296, 160000)


In [66]:
trials_pre_state_change = np.where(test.contra_data.trial_nums <=149)[0]
trials_post_state_change = np.where(test.contra_data.trial_nums >149)[0]

In [67]:
peaks = test.contra_data.trial_peaks
pre_peaks = [peaks[trial] for trial in trials_pre_state_change if peaks[trial].size > 0]
post_peaks = [peaks[trial] for trial in trials_post_state_change  if peaks[trial].size > 0]

In [68]:
# example mouse plot
fig, axs = plt.subplots(2,2)
heat_im = axs[0,0].imshow(test.contra_data.sorted_traces[ trials_pre_state_change,:], aspect='auto',
                            extent=[-10, 10, trials_pre_state_change.shape[0], 0], cmap='jet')
axs[0,0].scatter(test.contra_data.reaction_times[trials_pre_state_change],
                   np.arange(trials_pre_state_change.shape[0]) + 0.5, color='w', s=1)
heat_im = axs[1, 0].imshow(test.contra_data.sorted_traces[ trials_post_state_change,:], aspect='auto',
                            extent=[-10, 10, trials_post_state_change.shape[0], 0], cmap='jet')
axs[1,0].scatter(test.contra_data.reaction_times[trials_post_state_change],
                   np.arange(trials_post_state_change.shape[0]) + 0.5, color='w', s=1)
axs[0,0].set_xlim([-2,2])
axs[1,0].set_xlim([-2,2])
axs[0,0].axvline([0], color='k')
axs[1,0].axvline([0], color='k')


axs[0,1].plot(test.contra_data.time_points, np.mean(test.contra_data.sorted_traces[ trials_pre_state_change,:], axis=0))
axs[1,1].plot(test.contra_data.time_points, np.mean(test.contra_data.sorted_traces[ trials_post_state_change,:], axis=0))

In [69]:
font = {'size'   : 14}
matplotlib.rc('font', **font)

fig, axs1 = plt.subplots(1,1)
colours = cm.viridis(np.linspace(0.2, 0.7, 2))
pre_mean_trace = decimate(np.mean(test.contra_data.sorted_traces[trials_pre_state_change,:], axis=0), 10)
post_mean_trace = decimate(np.mean(test.contra_data.sorted_traces[ trials_post_state_change,:], axis=0), 10)
time_points = decimate(test.contra_data.time_points, 10)
traces = decimate(test.contra_data.sorted_traces, 10)
axs1.plot(time_points, pre_mean_trace, label='pre', color=colours[0])
pre_error_bar_lower, pre_error_bar_upper = calculate_error_bars(pre_mean_trace,
                                                        traces[trials_pre_state_change,:],
                                                        error_bar_method='ci')
                  
axs1.fill_between(time_points, pre_error_bar_lower, pre_error_bar_upper, alpha=0.4, linewidth=0, color=colours[0])
axs1.plot(time_points,post_mean_trace, label='post', color=colours[1])
post_error_bar_lower, post_error_bar_upper = calculate_error_bars(post_mean_trace,
                                                        traces[ trials_post_state_change,:],
                                                        error_bar_method='ci')
axs1.fill_between(time_points, post_error_bar_lower, post_error_bar_upper, alpha=0.4, linewidth=0, color=colours[1])
axs1.set_xlim([-2,2])
axs1.axvline([0], color='k')
axs1.set_ylabel('z-scored fluorescence')
axs1.set_xlabel('Time (s)')

axs1.spines['right'].set_visible(False)
axs1.spines['top'].set_visible(False)
plt.legend()
plt.tight_layout()
        


Bootstrapping...: 100%|████████████████████| 1000/1000 [00:14<00:00, 69.92it/s]


In [128]:
# group data
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
state_change_data_file = os.path.join(processed_data_dir, 'state_change_data_all_mice.csv')
all_session_change_data = pd.read_pickle(state_change_data_file)

In [129]:
pre_data = df1[df1['trial type']=='pre']['peak size']
post_data = df1[df1['trial type']=='post']['peak size']
stat, pval = stats.ttest_rel(pre_data, post_data)
pval *= .5 # multiply by .5 bc doing a one-sided t-test! we have a hypothesis that it's going a certain way 
pval

0.08588807590568354

In [156]:
df_for_plot = df1.pivot(index='trial type', columns='mouse', values='peak size').sort_values('trial type', ascending=False)

In [157]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, color='gray', legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)


In [159]:
fig, ax = plt.subplots()
francescas_plot(ax, df_for_plot, mean_line_color='#7FB5B5', mean_linewidth=6, show_err_bar=False)
plt.xticks([0, 1], ['pre state change', 'post state change'])
plt.ylabel('Z-scored fluorescence')

Text(0, 0.5, 'Z-scored fluorescence')

In [160]:
import os

filepath=os.path.join('W:\\photometry_2AC\\plots', 'pre_and_post_state_change.pdf' )
fig.savefig(filepath)

In [126]:
figure_dir = figure_dir = 'W:\\photometry_2AC\\plots\\SNL_photo21_26\\state_change_experiment'

font = {'size'   : 14}
matplotlib.rc('font', **font)

fig, axs = plt.subplots(1,1)
sns.lineplot(ax=axs, x='trial type', y='peak size', data=df1.sort_values('trial type', ascending=False), err_style='bars', sort=False)
axs.text(0.9, 1.9, 'p-value = {0:.3f}'.format(pval))



axs.spines['right'].set_visible(False)
axs.spines['top'].set_visible(False)
axs.set_ylabel('response (z-score)')
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, 'group_data_peaks_vs_state.pdf'))

In [105]:
df_only_behav_change_mice = df1[df1.mouse != 'SNL_photo22']

In [106]:
pre_data = df_only_behav_change_mice[df_only_behav_change_mice['trial type']=='pre']['peak size']
post_data = df_only_behav_change_mice[df_only_behav_change_mice['trial type']=='post']['peak size']

stat, pval = stats.ttest_rel(pre_data, post_data)
pval *= .5 # multiply by .5 bc doing a one-sided t-test! we have a hypothesis that it's going a certain way 
pval

0.10408651669490764